In [2]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import numpy as np

In [ ]:
# define the scope and credentials
scope =[
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(
    'sales-sync-automation-dbe6b9c8399b.json', 
    scope
)

#authorize the client
client = gspread.authorize(creds)

# Define the Google Sheet and worksheet
mySheet = "Sales Sync Test"

In [86]:
# Open the entire spreadsheet file
sh = client.open(mySheet)  # This gives you access to tabs

# open the Google Sheet
sheet = client.open(mySheet).sheet1

In [11]:
# open the csv file and make it a dataframe
df = pd.read_csv('sales_bulk_dirty.csv')

In [12]:
# Clean the column names (strip spaces, capitalize)
df.columns = [col.strip().capitalize() for col in df.columns]

In [13]:
df

,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount
0,24/06/2025,Matte Kiss,Lipstain,1.0,1126,Shopify,South,NaN
1,2025-05-01,Lash Pro,Lash,5.0,5325,Whatsapp,North,NaN
2,20/06/2025,StickTight,Glue,NaN,0,Facebook,West,NaN
3,2025-05-02,Matte Kiss,Lipstain,7.0,9366,Site,South,NaN
4,06/10/25,Lip Pop,Lipstain,NaN,0,Instagram,South,NaN
...,...,...,...,...,...,...,...,...
495,18/05/2025,GlueMax,Glue,NaN,0,Site,East,NaN
496,06/06/25,Lash Queen,Lash,6.0,7566,Facebook,South,NaN
497,06/01/25,GlueMax,Glue,6.0,6066,WhatsApp,North,16%
498,06/03/25,StickTight,Glue,5.0,9415,Shopify,North,16%


In [16]:
# See how many rows failed to parse
df['Parsed_Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)


# Display the rows where date parsing failed
bad_dates = df[df['Parsed_Date'].isna()]

# Display the number of bad date rows and the first few rows
print(f"Bad date rows: {len(bad_dates)}")
display(bad_dates.head())


Bad date rows: 347


,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount,Parsed_Date
1,2025-05-01,Lash Pro,Lash,5.0,5325,Whatsapp,North,NaN,NaT
3,2025-05-02,Matte Kiss,Lipstain,7.0,9366,Site,South,NaN,NaT
4,06/10/25,Lip Pop,Lipstain,NaN,0,Instagram,South,NaN,NaT
7,05/15/25,Lip Pop,Lipstain,NaN,0,Shopify,West,NaN,NaT
8,05/28/25,StickTight,Glue,NaN,0,Shopify,South,20%,NaT


In [19]:
# Strip whitespace and normalize separators
df['Date'] = df['Date'].astype(str).str.strip().str.replace(r"[^\d/-]", "", regex=True)


In [ ]:
from dateutil.parser import parse

def smart_parse(date_str):
    try:
        # Try day-first first (common format)
        return parse(date_str, dayfirst=True)
    except:
        try:
            # Fallback to month-first
            return parse(date_str, dayfirst=False)
        except:
            return pd.NaT

# Apply to the column
df['Parsed_Date'] = df['Date'].apply(smart_parse)

In [25]:
print("Rows with good dates:", df['Parsed_Date'].notna().sum())
print("Still broken:", df['Parsed_Date'].isna().sum())


Rows with good dates: 500
Still broken: 0


In [26]:
# Update the 'Date' column with parsed dates
df['Date'] = df['Parsed_Date']

# Drop the temporary 'Parsed_Date' column
df.drop(columns=['Parsed_Date'], inplace=True)

# Format date to yyyy-mm-dd for Sheets
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')


In [35]:
df

,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount
0,2025-06-24,Matte Kiss,Lipstain,1.0,1126,Shopify,South,NaN
1,2025-01-05,Lash Pro,Lash,5.0,5325,Whatsapp,North,NaN
2,2025-06-20,StickTight,Glue,NaN,0,Facebook,West,NaN
3,2025-02-05,Matte Kiss,Lipstain,7.0,9366,Site,South,NaN
4,2025-10-06,Lip Pop,Lipstain,NaN,0,Instagram,South,NaN
...,...,...,...,...,...,...,...,...
495,2025-05-18,GlueMax,Glue,NaN,0,Site,East,NaN
496,2025-06-06,Lash Queen,Lash,6.0,7566,Facebook,South,NaN
497,2025-01-06,GlueMax,Glue,6.0,6066,WhatsApp,North,16%
498,2025-03-06,StickTight,Glue,5.0,9415,Shopify,North,16%


In [39]:
# Strip out junk and ensure it's numeric
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

# Replace NaNs with 0 and convert to integer
df['Quantity'] = df['Quantity'].fillna(0).astype(int)

# Optional: sanity check
print(df['Quantity'].describe())


count    500.000000
mean       2.778000
std        3.433408
min        0.000000
25%        0.000000
50%        1.000000
75%        6.000000
max       10.000000
Name: Quantity, dtype: float64


In [41]:
# No negative quantities allowed!
df = df[df['Quantity'] >= 0]
df

,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount
0,2025-06-24,Matte Kiss,Lipstain,1,1126,Shopify,South,NaN
1,2025-01-05,Lash Pro,Lash,5,5325,Whatsapp,North,NaN
2,2025-06-20,StickTight,Glue,0,0,Facebook,West,NaN
3,2025-02-05,Matte Kiss,Lipstain,7,9366,Site,South,NaN
4,2025-10-06,Lip Pop,Lipstain,0,0,Instagram,South,NaN
...,...,...,...,...,...,...,...,...
495,2025-05-18,GlueMax,Glue,0,0,Site,East,NaN
496,2025-06-06,Lash Queen,Lash,6,7566,Facebook,South,NaN
497,2025-01-06,GlueMax,Glue,6,6066,WhatsApp,North,16%
498,2025-03-06,StickTight,Glue,5,9415,Shopify,North,16%


In [53]:
# Strip weird formatting, ensure numeric
df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')

# Fill missing with 0, convert to int
df['Revenue'] = df['Revenue'].fillna(0).astype(int)

# Optional: remove negative revenue if not allowed
df = df[df['Revenue'] >= 0]


In [55]:
# Check for suspicious rows
sus = df[(df['Quantity'] == 0) & (df['Revenue'] > 0)]
print(f"Suspicious rows: {len(sus)}")
display(sus.head())


Suspicious rows: 0


,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount


In [58]:
# Check unique values in 'Channel' column
df['Channel'].unique()

array(['Shopify', 'Whatsapp', 'Facebook', 'Site', 'Instagram', 'Insta',
       'IG', 'WhatsApp'], dtype=object)

In [59]:
# Lowercase and strip whitespace
df['Channel'] = df['Channel'].astype(str).str.strip().str.lower()

# Map known aliases to standard names
channel_map = {
    'insta': 'Instagram',
    'ig': 'Instagram',
    'instagram': 'Instagram',
    'whatsapp': 'WhatsApp',
    'whatsApp': 'WhatsApp',
    'site': 'Shopify',
    'shopify': 'Shopify',
    'facebook': 'Facebook',
    'fb': 'Facebook'
}

# Apply the map
df['Channel'] = df['Channel'].replace(channel_map)

# Optional: Capitalize everything consistently
df['Channel'] = df['Channel'].str.title()


In [61]:
# Check once again for unique values
df['Channel'].unique()

array(['Shopify', 'Whatsapp', 'Facebook', 'Instagram'], dtype=object)

In [64]:
df['Region'].unique()

array(['South', 'North', 'West', 'East'], dtype=object)

In [66]:
df['Discount'].unique()

array([nan, '16%', '17%', '20%', '11%', '19%', '7%', '9%', '28%', '29%',
       '8%', '21%', '14%', '12%', '23%', '13%', '6%', '18%', '27%', '24%',
       '10%', '30%', '22%', '25%', '15%', '26%', '5%'], dtype=object)

In [67]:
# Remove '%' and whitespace, convert to number
df['Discount'] = df['Discount'].astype(str).str.replace('%', '').str.strip()

# Convert to numeric, fill NaNs with 0, and convert to integer
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce').fillna(0).astype(int)
df['Discount']

0       0
1       0
2       0
3       0
4       0
       ..
495     0
496     0
497    16
498    16
499     0
Name: Discount, Length: 500, dtype: int64

In [69]:
# At this point, we will be adding a new column for discount rate and calculating the discount amount and net revenue.

# Convert Discount to decimal
df['Discount_Rate'] = df['Discount'] / 100

# Calculate discount amount
df['Discount_Amount'] = (df['Revenue'] * df['Discount_Rate']).round(2)

# Calculate net revenue
df['Net_Revenue'] = (df['Revenue'] - df['Discount_Amount']).round(2)

df.head(70)


,Date,Product,Category,Quantity,Revenue,Channel,Region,Discount,Discount_Rate,Discount_Amount,Net_Revenue
0,2025-06-24,Matte Kiss,Lipstain,1,1126,Shopify,South,0,0.00,0.00,1126.00
1,2025-01-05,Lash Pro,Lash,5,5325,Whatsapp,North,0,0.00,0.00,5325.00
2,2025-06-20,StickTight,Glue,0,0,Facebook,West,0,0.00,0.00,0.00
3,2025-02-05,Matte Kiss,Lipstain,7,9366,Shopify,South,0,0.00,0.00,9366.00
4,2025-10-06,Lip Pop,Lipstain,0,0,Instagram,South,0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
65,2025-10-06,Lash Pro,Lash,6,9342,Shopify,West,0,0.00,0.00,9342.00
66,2025-05-05,Lip Pop,Lipstain,3,3939,Instagram,West,27,0.27,1063.53,2875.47
67,2025-05-17,Matte Kiss,Lipstain,0,0,Whatsapp,North,0,0.00,0.00,0.00
68,2025-05-28,Lash Pro,Lash,4,4996,Shopify,East,0,0.00,0.00,4996.00


In [ ]:
total_revenue = df['Revenue'].sum()
print(f"Total Revenue: ${total_revenue:,}")


💵 Total Revenue: $2,095,395


In [73]:
net_revenue = df['Net_Revenue'].sum()
print(f"Net Revenue (Total Revenue minus Discount): ${net_revenue:,}")


Net Revenue (Total Revenue minus Discount): $1,920,932.5


In [74]:
total_units = df['Quantity'].sum()
print(f"Total Units Sold: {total_units}")


Total Units Sold: 1389


In [75]:
avg_discount = df['Discount'].mean().round(2)
print(f"Average Discount: {avg_discount}%")


Average Discount: 8.01%


In [97]:
top_channel = df['Channel'].mode()[0]
print(f"Top Channel: {top_channel}")


Top Channel: Instagram


In [96]:
top_product = df.groupby('Product')['Quantity'].sum().sort_values(ascending=False).head(1)
print("Top Product:")
display(top_product)


Top Product:


Product
GlueMax    287
Name: Quantity, dtype: int64

In [ ]:
# Now we will upload the cleaned data to Google Sheets

# Clear existing content
sheet.clear()

# Prepare data
data_to_upload = [df.columns.values.tolist()] + df.values.tolist()

# Update sheet
sheet.update(data_to_upload)


{'spreadsheetId': '14QZDd9AMxqYkduaEBpvXzIKusbC8054XG5ve69WWV-0',
 'updatedRange': 'Sheet1!A1:K501',
 'updatedRows': 501,
 'updatedColumns': 11,
 'updatedCells': 5511}

In [87]:
# Adding the summary statistics to a different tab of the same Google Sheet

# Check if summary tab exists
try:
    summary_ws = sh.worksheet("Summary")
except:
    summary_ws = sh.add_worksheet(title="Summary", rows="20", cols="3")


In [98]:
# Prepare summary data
summary_data = [
    ['Metric', 'Value'],
    ['Total Revenue', f"${total_revenue:,}"],
    ['Net Revenue', f"${net_revenue:,}"],
    ['Total Units Sold', f"{total_units} Units"],
    ['Average Discount (%)', f'{avg_discount}%'],
    ['Top Channel', top_channel],
    ['Top Product', top_product.index[0]]
]

#Always remember to parse everything to string before uploading to Google Sheets to avoid type errors


In [99]:
# Clear it first
summary_ws.clear()

# Upload all rows
summary_ws.update(values=summary_data, range_name='A1')

{'spreadsheetId': '14QZDd9AMxqYkduaEBpvXzIKusbC8054XG5ve69WWV-0',
 'updatedRange': 'Summary!A1:B7',
 'updatedRows': 7,
 'updatedColumns': 2,
 'updatedCells': 14}